In [17]:
%load_ext autoreload
%autoreload 2
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '2'

from main import parse_args
import argparse
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import numpy as np

from dataset import Dictionary, VQAFeatureDataset, PadCollate
import base_model
from train import train
import utils

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
parser = parse_args()
args = parser.parse_args([])
args.epochs = 30
args.batch_size = 512
print(args)

torch.manual_seed(args.seed)
torch.cuda.manual_seed(args.seed)
torch.backends.cudnn.benchmark = True

dictionary = Dictionary.load_from_file('data/dictionary.pkl')

Namespace(batch_size=512, epochs=30, model='baseline0_newatt', num_hid=1024, output='tmp', seed=1111)
loading dictionary from data/dictionary.pkl


In [3]:
from dataset import VQAFeatureDataset
train_dset = VQAFeatureDataset('train', dictionary)
eval_dset = VQAFeatureDataset('val', dictionary)

  0%|          | 124/82783 [00:00<01:06, 1233.83it/s]

loading features from numpy files


 58%|█████▊    | 48280/82783 [00:48<00:34, 1003.86it/s]

Not exist: bottom-up-adaptK/trainval/337663.npy


  0%|          | 158/40504 [00:00<00:25, 1578.66it/s]

loading features from numpy files


 24%|██▎       | 9599/40504 [00:07<00:22, 1363.65it/s]

no features:27642


100%|██████████| 40504/40504 [00:28<00:00, 1428.61it/s]


In [4]:
batch_size = args.batch_size
train_loader = DataLoader(train_dset, batch_size, shuffle=True, num_workers=4, collate_fn=PadCollate(dim=0))
eval_loader =  DataLoader(eval_dset, batch_size, shuffle=True, num_workers=4, collate_fn=PadCollate(dim=0))

In [18]:
constructor = 'build_%s' % args.model
model = getattr(base_model, constructor)(train_dset, args.num_hid).cuda()
model.w_emb.init_embedding('data/glove6b_init_300d.npy')

model = nn.DataParallel(model).cuda()

In [35]:
#del model
torch.cuda.empty_cache()
#print(torch.cuda.memory_cached())
#torch.cuda.memory_allocated()
#torch.cuda.memory_cached(device=None)

In [19]:
train(model, train_loader, eval_loader, args.epochs, args.output)


  2%|▏         | 21/867 [00:25<17:07,  1.21s/it]Process Process-253:
Process Process-256:
Process Process-255:
Process Process-254:
Exception NameError: "global name 'FileNotFoundError' is not defined" in <bound method _DataLoaderIter.__del__ of <torch.utils.data.dataloader._DataLoaderIter object at 0x7fdb87431210>> ignored


KeyboardInterrupt: 

Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/qingli/anaconda2/lib/python2.7/multiprocessing/process.py", line 267, in _bootstrap
  File "/home/qingli/anaconda2/lib/python2.7/multiprocessing/process.py", line 267, in _bootstrap
  File "/home/qingli/anaconda2/lib/python2.7/multiprocessing/process.py", line 267, in _bootstrap
  File "/home/qingli/anaconda2/lib/python2.7/multiprocessing/process.py", line 267, in _bootstrap
    self.run()
    self.run()
    self.run()
    self.run()
  File "/home/qingli/anaconda2/lib/python2.7/multiprocessing/process.py", line 114, in run
  File "/home/qingli/anaconda2/lib/python2.7/multiprocessing/process.py", line 114, in run
  File "/home/qingli/anaconda2/lib/python2.7/multiprocessing/process.py", line 114, in run
  File "/home/qingli/anaconda2/lib/python2.7/multiprocessing/process.py", line 114, in run
    self._target(*self._args, **self._kwargs

# Train an ensemble of models with different seeds

In [10]:
%load_ext autoreload
%autoreload 2
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '2'

from main import parse_args
import argparse
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import numpy as np

from dataset import Dictionary, VQAFeatureDataset, PadCollate
import base_model
import utils

from train_all import train, evaluate

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
parser = parse_args()
args = parser.parse_args([])
args.epochs = 15

img_feat_name = 'cls_prob'
dictionary = Dictionary.load_from_file('data/dictionary.pkl')
train_dset = VQAFeatureDataset('train', dictionary, img_feat_name)
eval_dset = VQAFeatureDataset('val', dictionary, img_feat_name)
test_dset = VQAFeatureDataset('test', dictionary, img_feat_name)
batch_size = args.batch_size
train_loader =  DataLoader(train_dset + eval_dset, batch_size, shuffle=True, num_workers=1, collate_fn=PadCollate(dim=0))
test_loader =  DataLoader(test_dset, batch_size, shuffle=False, num_workers=0, collate_fn=PadCollate(dim=0))

loading dictionary from data/dictionary.pkl


  0%|          | 2/82783 [00:00<1:28:14, 15.64it/s]

loading features from numpy files


 58%|█████▊    | 48038/82783 [10:10<07:21, 78.75it/s]

Not exist: bottom-up-adaptK/trainval/337663.npy


...

 23%|██▎       | 9367/40504 [01:53<06:15, 82.87it/s]

cls_prob not in bottom-up-adaptK/trainval/27642.npy


  0%|          | 1/81434 [00:00<2:27:58,  9.17it/s]

loading features from numpy files


100%|██████████| 81434/81434 [16:29<00:00, 82.30it/s]


In [3]:
import numpy as np
import os
n_models = 10
pred_list_sum = 0
for idx in range(n_models):
    print(idx)
    args.seed = idx
    args.output = 'saved_models/exp%d'%idx
    
    torch.manual_seed(args.seed)
    torch.cuda.manual_seed(args.seed)
    torch.backends.cudnn.benchmark = True

    constructor = 'build_%s' % args.model
    model = getattr(base_model, constructor)(train_dset, args.num_hid).cuda()
    model.w_emb.init_embedding('data/glove6b_init_300d.npy')
    model = nn.DataParallel(model).cuda()

    train(model, train_loader, test_loader, args.epochs, args.output)
    q_id_list, pred_list = evaluate(model, test_loader)
    pred_list_sum += pred_list

out_prob = 'saved_models/test2015'
np.save(out_prob, pred_list)
out_prob = 'saved_models/test2015_sum'
np.save(out_prob, pred_list_sum)

0


  0%|          | 0/1286 [00:00<?, ?it/s]

epoch 0/15


  0%|          | 0/1286 [00:00<?, ?it/s]

epoch 1/15


...

  0%|          | 0/1286 [00:00<?, ?it/s]

epoch 13/15


  0%|          | 0/1286 [00:00<?, ?it/s]

epoch 14/15


100%|██████████| 875/875 [02:30<00:00,  5.80it/s]


In [11]:
import cPickle as pkl
from tqdm import tqdm
import json
label2ans = pkl.load(open('data/cache/trainval_label2ans.pkl'))
results = []
for x, y in zip(q_id_list, np.argmax(pred_list_sum, 1)):
    y = label2ans[y]
    results.append({'question_id': x, 'answer': y})
json.dump(results, open('saved_models/test2015_predictions.json', 'w'))

In [12]:
q_id2result = {x['question_id']:x for x in results}
test_dev_set = json.load(open('data/v2_OpenEnded_mscoco_test-dev2015_questions.json'))['questions']
test_dev_q_ids = [x['question_id'] for x in test_dev_set]
results_test_dev = [q_id2result[x] for x in test_dev_q_ids]
json.dump(results_test_dev, open('saved_models/test-dev_predictions.json', 'w'))

# Test models

In [13]:
%load_ext autoreload
%autoreload 2
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '2'

from main import parse_args
import argparse
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import numpy as np

from dataset import Dictionary, VQAFeatureDataset, PadCollate
import base_model
import utils

from train_all import train, evaluate

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
parser = parse_args()
args = parser.parse_args([])
args.epochs = 15

img_feat_name = 'cls_prob'
dictionary = Dictionary.load_from_file('data/dictionary.pkl')
#train_dset = VQAFeatureDataset('train', dictionary, img_feat_name)
#eval_dset = VQAFeatureDataset('val', dictionary, img_feat_name)
test_dset = VQAFeatureDataset('test', dictionary, img_feat_name)
batch_size = args.batch_size
#train_loader =  DataLoader(train_dset + eval_dset, batch_size, shuffle=True, num_workers=1, collate_fn=PadCollate(dim=0))
test_loader =  DataLoader(test_dset, batch_size, shuffle=False, num_workers=0, collate_fn=PadCollate(dim=0))

loading dictionary from data/dictionary.pkl


  0%|          | 8/81434 [00:00<17:41, 76.70it/s]

loading features from numpy files


100%|██████████| 81434/81434 [14:52<00:00, 91.23it/s]


In [20]:
import numpy as np
from tqdm import trange
import os
n_models = 10
pred_list_sum = 0
for idx in trange(n_models):
    print(idx)
    args.seed = idx
    args.output = 'saved_models/exp%d'%idx
    args.init_from = os.path.join(args.output, 'model.pth')

    constructor = 'build_%s' % args.model
    model = getattr(base_model, constructor)(test_dset, args.num_hid).cuda()
    model.w_emb.init_embedding('data/glove6b_init_300d.npy')
    model = nn.DataParallel(model).cuda()

    print('Init from: ' + args.init_from)
    init_model = torch.load(args.init_from)
    model.load_state_dict(init_model)
    
    q_id_list, pred_list = evaluate(model, test_loader)
    pred_list_sum += pred_list

out_prob = 'saved_models/test2015'
np.save(out_prob, pred_list)
out_prob = 'saved_models/test2015_sum'
np.save(out_prob, pred_list_sum)

  0%|          | 0/10 [00:00<?, ?it/s]

0
Init from: saved_models/exp0/model.pth


 10%|█         | 1/10 [17:48<2:40:16, 1068.52s/it]

1
Init from: saved_models/exp1/model.pth


 20%|██        | 2/10 [33:41<2:14:45, 1010.65s/it]

...

 80%|████████  | 8/10 [1:30:30<22:37, 678.76s/it]

8
Init from: saved_models/exp8/model.pth


 90%|█████████ | 9/10 [1:33:11<10:21, 621.33s/it]

9
Init from: saved_models/exp9/model.pth


100%|██████████| 10/10 [1:35:53<00:00, 575.36s/it]


In [21]:
import cPickle as pkl
from tqdm import tqdm
import json
label2ans = pkl.load(open('data/cache/trainval_label2ans.pkl'))
results = []
for x, y in zip(q_id_list, np.argmax(pred_list_sum, 1)):
    y = label2ans[y]
    results.append({'question_id': x, 'answer': y})
json.dump(results, open('saved_models/test2015_predictions.json', 'w'))

q_id2result = {x['question_id']:x for x in results}
test_dev_set = json.load(open('data/v2_OpenEnded_mscoco_test-dev2015_questions.json'))['questions']
test_dev_q_ids = [x['question_id'] for x in test_dev_set]
results_test_dev = [q_id2result[x] for x in test_dev_q_ids]
json.dump(results_test_dev, open('saved_models/test-dev_predictions.json', 'w'))

# Fusion

In [6]:
import os, json
question_path = os.path.join('data', 'v2_OpenEnded_mscoco_test2015_questions.json')
questions = sorted(json.load(open(question_path))['questions'],
                           key=lambda x: x['question_id'])
q_id_list = [x['question_id'] for x in questions]

In [1]:
import numpy as np
attr_pred_list_sum = np.load('../adaptK-bottom-up-attention-vqa-attr/saved_models/test2015_sum.npy')
cls_pred_list_sum = np.load('../adaptK-bottom-up-attention-vqa-cls/saved_models/test2015_sum.npy')
feat_pred_list_sum = np.load('../adaptK-bottom-up-attention-vqa/saved_models/test2015_sum.npy')

In [10]:
pred_list_sum = 0.1 * (attr_pred_list_sum/10) + 0.1 * (cls_pred_list_sum/10) + 0.8 * (feat_pred_list_sum/13)

In [11]:
import cPickle as pkl
from tqdm import tqdm
import json
label2ans = pkl.load(open('data/cache/trainval_label2ans.pkl'))
results = []
for x, y in zip(q_id_list, np.argmax(pred_list_sum, 1)):
    y = label2ans[y]
    results.append({'question_id': x, 'answer': y})
json.dump(results, open('test2015_predictions.json', 'w'))

q_id2result = {x['question_id']:x for x in results}
test_dev_set = json.load(open('data/v2_OpenEnded_mscoco_test-dev2015_questions.json'))['questions']
test_dev_q_ids = [x['question_id'] for x in test_dev_set]
results_test_dev = [q_id2result[x] for x in test_dev_q_ids]
json.dump(results_test_dev, open('test-dev_predictions.json', 'w'))